In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

test=pd.read_csv("/kaggle/input/movie-recommender-system-2022/test.csv")
movies=pd.read_csv("/kaggle/input/movie-recommender-system-2022/movies.csv")
ratings=pd.read_csv("/kaggle/input/movie-recommender-system-2022/raitings_training.csv")
users=pd.read_csv("/kaggle/input/movie-recommender-system-2022/users.csv")


def get_users_movie(itemid):
    users = ratings[ratings['itemid']==itemid]['userid'].keys()
    return users

def get_movies_user(userid):
    movies = ratings[ratings['userid']==userid]['itemid'].keys()
    return movies

"""Función de Similitud entre peliculas
    Correlación de Pearson"""
def movie_similarity(movie,movie_mean,itemid,itemid_mean):
    """
    movie: una de las peliculas evaluadas por el usuario
    itemid: pelicula a evaluar
    """

    
    #Usuarios que valoran la pelicula
    users_itemid = get_users_movie(int(itemid))
    users_itemid_list=[]
    for u in users_itemid: #Guardamos usuarios que han visto
        user_itm=ratings['userid'].iloc[u]
        users_itemid_list.append(user_itm)
    
    #Usuarios que valoran la pelicula evaluada por el usuario
    users_movie = get_users_movie(int(movie))
    
    
    numerador = 0
    denomi1 = 0
    denomi2 = 0 
    for user in users_movie:
        #Sacamos el id de usuario
        uid=ratings['userid'].iloc[user] 

        if uid in users_itemid_list: #Comprobamos que haya valorado ambas peliculas (la de busqueda, y la del usuario )
            

            #Obtenemos la valoracion del user en el item y en la peli actual
            valoracion_movie=ratings[(ratings['itemid']==int(movie))&(ratings['userid']==int(uid))]['rating'].keys() 
            valoracion_itemid=ratings[(ratings['itemid']==int(itemid))&(ratings['userid']==int(uid))]['rating'].keys() 



            #Obtenemos el rating de ambas peliculas la de busqueda y la del usuario
            for i in valoracion_movie:                
                rating_movie=ratings['rating'].iloc[i] 


            for i in valoracion_itemid:
                rating_itemid=ratings['rating'].iloc[i] 


            #Calculamos numero y denominador del coeficiciente correlacion de Pearson
            numerador += (rating_itemid-itemid_mean) * (rating_movie-movie_mean)
            denomi1 += pow(rating_itemid-itemid_mean,2)
            denomi2 += pow(rating_movie-movie_mean,2) 
            
        
    denomi = np.sqrt(denomi1)* np.sqrt(denomi2)

    #calculacmos el coeficciente correlacion de Pearson
    if denomi==0:
        coef=0
    else:
        coef = numerador/denomi
    

    if np.isnan(coef): # suposición en caso de indeterminación
        return 0
    return coef


#Cramos un dict para almacenar los resultado
dicti={}


#Recorremos los casos del archivo
for i in range(len(test)):
    id = test['Id'][i]
    
    userid, itemid = id.split("-")
    
    #Peliculas valoradas por el usuario
    movies_user =  get_movies_user(int(userid))
    
    #Usuarios que valoran la pelicula
    users_movie = get_users_movie(int(itemid))
    
    #genero pelicula"
    gen_itemid=movies[movies['itemid']==int(itemid)]['genre'] 
    for g in gen_itemid:    
        g = str(g)
        generos_itemid=g.split('|')#Si existen generos los guarda y separa
            
    #Valoracion media  pelicula
    itemid_mean=ratings[ratings['itemid']==int(itemid)]['rating'].mean() 
    
    #Ponemos un maximo para solo buscar en el 10% de la peliculas
    max_cont=0.1*len(movies_user)
    cont=0
    
    coef_sum= 0
    coef_sum_rat =0
    
    #Recorremos peliculas vistas por el usuario
    for movie in movies_user:

        
        gen=movies[movies['itemid']==int(movie)]['genre'] 
        generos=[]
        for g in gen:   
            g = str(g)
            generos=g.split('|')#Si existen generos los guarda y separa
        #print(generos_itemid)
        #print(generos)
        comun = set(generos_itemid).intersection(generos) #Comprobamos que tenga genero en comun 
        if comun :
            
            movie_mean=ratings[ratings['itemid']==int(movie)]['rating'].mean()
            coef = movie_similarity(movie,movie_mean,itemid,itemid_mean)
            if coef >=0:

                #Vemos la media del usuario en la movie actual
                rating_movie_userid=ratings[(ratings['itemid']==int(movie))&(ratings['userid']==int(userid))]['rating'].keys()
                for i in rating_movie_userid:
                    #Obtenemos el rating
                    rating_movie_user=ratings['rating'].iloc[i]

                    #Calculamos la suma de los coeficientes para realizar la media ponderada de los productos que el usuario evaluo
                    coef_sum_rat+=coef*rating_movie_user
                    coef_sum +=coef
                    

                    #Aumentamos el contador y comprobamos si hemos pasado el maximo
                    cont+=1
                    if cont>=max_cont: 
                        print("contador",max_cont)
                        break;
    
    #Calculamos la valoracion del usario en base a los coeficientes almacenados
    
    if coef_sum == 0 : #dvision por cero
        coef_sum = 1
    rating_final = coef_sum_rat/coef_sum
    if np.isnan(rating_final): # suposición en caso de indeterminación
        rating_final = 0
    
   
    dicti[id]= rating_final


#Mostramos los rrsultados
archivo = pd.read_csv('/kaggle/input/movie-recommender-system-2022/test.csv',dtype=('string_'),delimiter=',', header=0)
archivo['Expected'] = dicti.values()
archivo.to_csv('submission.csv', header = True, index = False)

/kaggle/input/movie-recommender-system-2022/sample_submission.csv
/kaggle/input/movie-recommender-system-2022/movies.csv
/kaggle/input/movie-recommender-system-2022/raitings_training.csv
/kaggle/input/movie-recommender-system-2022/users.csv
/kaggle/input/movie-recommender-system-2022/test.csv
